# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9ce42e3760>
├── 'a' --> tensor([[-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862]])
└── 'x' --> <FastTreeValue 0x7f9ce42e3fd0>
    └── 'c' --> tensor([[ 0.8780,  1.7460, -0.1844,  1.0019],
                        [-0.3078, -1.4227, -0.1423, -0.8636],
                        [-0.6708, -1.4015, -1.2057, -0.3303]])

In [4]:
t.a

tensor([[-1.4750, -1.3367,  0.0080],
        [-0.2362, -0.4234, -0.8862]])

In [5]:
%timeit t.a

61.5 ns ± 0.0602 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9ce42e3760>
├── 'a' --> tensor([[-0.7074,  0.8171,  0.6072],
│                   [-0.2061,  1.9341, -0.3205]])
└── 'x' --> <FastTreeValue 0x7f9ce42e3fd0>
    └── 'c' --> tensor([[ 0.8780,  1.7460, -0.1844,  1.0019],
                        [-0.3078, -1.4227, -0.1423, -0.8636],
                        [-0.6708, -1.4015, -1.2057, -0.3303]])

In [7]:
%timeit t.a = new_value

62.7 ns ± 0.0233 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862]]),
    x: Batch(
           c: tensor([[ 0.8780,  1.7460, -0.1844,  1.0019],
                      [-0.3078, -1.4227, -0.1423, -0.8636],
                      [-0.6708, -1.4015, -1.2057, -0.3303]]),
       ),
)

In [10]:
b.a

tensor([[-1.4750, -1.3367,  0.0080],
        [-0.2362, -0.4234, -0.8862]])

In [11]:
%timeit b.a

60 ns ± 0.0127 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2016,  0.0427, -1.9808],
               [ 0.8172,  0.7630,  0.0601]]),
    x: Batch(
           c: tensor([[ 0.8780,  1.7460, -0.1844,  1.0019],
                      [-0.3078, -1.4227, -0.1423, -0.8636],
                      [-0.6708, -1.4015, -1.2057, -0.3303]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 42 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 661 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9ce42d4100>
├── 'a' --> tensor([[[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]],
│           
│                   [[-1.4750, -1.3367,  0.0080],
│                    [-0.2362, -0.4234, -0.8862]]])
└── 'x' --> <FastTreeValue 0x7f9c18bc8280>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 22 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9c1888ad30>
├── 'a' --> tensor([[-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862],
│                   [-1.4750, -1.3367,  0.0080],
│                   [-0.2362, -0.4234, -0.8862]])
└── 'x' --> <FastTreeValue 0x7f9c1888abe0>
    └── 'c' --> tensor([[ 0.8780,  1.7460, -0.1844,  1.0019],
                        [-0.3078, -1.4227, -0.1423, -0.8636],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 65.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

66.9 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]],
       
               [[-1.4750, -1.3367,  0.0080],
                [-0.2362, -0.4234, -0.8862]]]),
    x: Batch(
           c: tensor([[[ 0.8780,  1.7460, -0.1844,  1.0019],
                       [-0.3078, -1.4227, -0.1423, -0.8636],
                       [-0.6708, -1.4015, -1.2057, -0.3303]],
         

In [26]:
%timeit Batch.stack(batches)

79.4 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862],
               [-1.4750, -1.3367,  0.0080],
               [-0.2362, -0.4234, -0.8862]]),
    x: Batch(
           c: tensor([[ 0.8780,  1.7460, -0.1844,  1.0019],
                      [-0.3078, -1.4227, -0.1423, -0.8636],
                      [-0.6708, -1.4015, -1.2057, -0.3303],
                      [ 0.8780,  1.7460, -0.1844,  1.0019],
                      [-0.3078, -

In [28]:
%timeit Batch.cat(batches)

147 µs ± 383 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
